In [4]:
# Import necessary libraries
import numpy as np
from scipy import signal, interpolate, stats
import matplotlib.pyplot as plt

# Define functions for each step of the flowchart

def boxcar_filtering(input_spectrum, filter_size):
    """
    Apply boxcar filter on the input spectrum.
    """
    box = signal.boxcar(filter_size)
    filtered_spectrum = signal.convolve(input_spectrum, box, mode='same') / filter_size
    return filtered_spectrum

def difference_of_gaussians(spectrum1, spectrum2):
    """
    Compute the difference of two given spectra.
    """
    return spectrum1 - spectrum2

def find_local_minima(difference_spectrum):
    """
    Find local minima in the given spectrum.
    """
    minima = signal.argrelextrema(difference_spectrum, np.less)
    return minima

def linear_interpolation(features):
    """
    Perform linear interpolation between given features.
    """
    x = np.arange(len(features))
    f = interpolate.interp1d(x, features, fill_value="extrapolate")
    return f

def find_local_maxima(difference_spectrum):
    """
    Find local maxima in the given spectrum.
    """
    maxima = signal.argrelextrema(difference_spectrum, np.greater)
    return maxima

def check_zero_crossings(preliminary_hull):
    """
    Check for zero crossings in the preliminary hull.
    """
    zero_crossings = np.where(np.diff(np.sign(preliminary_hull)))[0]
    return zero_crossings

# Let's use a synthetic signal as the input spectrum
input_spectrum = np.sin(3 * np.pi * np.linspace(0, 1, 201))

# 1. Successive filtering of input spectrum with 2% and 10% boxcar filter
filter_size_2_percent = int(0.02 * len(input_spectrum))
filter_size_10_percent = int(0.1 * len(input_spectrum))

filtered_spectrum_2_percent = boxcar_filtering(input_spectrum,
                                               filter_size_2_percent)
filtered_spectrum_10_percent = boxcar_filtering(input_spectrum,
                                                filter_size_10_percent)

# 2. Calculate the difference of gaussians of the two resulting pairs from the filtering step
difference_spectrum = difference_of_gaussians(filtered_spectrum_2_percent,
                                              filtered_spectrum_10_percent)



# 3. Take Gaussian difference result with the largest standard deviation 
# and divide the first filtering step result by the second smoothing result to extract the local minima, 
# which are preliminary absorption features
if np.std(difference_spectrum) > np.std(filtered_spectrum_2_percent):
    absorption_features = filtered_spectrum_2_percent / difference_spectrum
else:
    absorption_features = filtered_spectrum_10_percent / difference_spectrum

local_minima = find_local_minima(absorption_features)

# 4. Linear interpolation between preliminary features
f_minima = linear_interpolation(local_minima[0])

# 5. Calculate difference between input spectrum and linear interpolation of minimas 
# and find maximum in each of the segments with zero separation
difference_spectrum = input_spectrum - f_minima(np.arange(len(input_spectrum)))
local_maxima = find_local_maxima(difference_spectrum)

# # 6. Linear interpolation between maxima gives preliminary hull
# f_maxima = linear_interpolation(local_maxima)

# # 7. Iterative check whether the preliminary hull contains zero crossings, if so then add new continuum points
# zero_crossings = check_zero_crossings(f_maxima(np.arange(len(input_spectrum))))

# # 8. Geometric hull

# geometric_hull = f_maxima(np.arange(len(input_spectrum)))

In [1]:
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from scipy.ndimage import filters

# def geometric_hull(input_spectrum):
#     # Step 1: Successive filtering of input spectrum with 2% and 10% boxcar filter
#     boxcar_2 = signal.boxcar(int(len(input_spectrum) * 0.02))
#     boxcar_10 = signal.boxcar(int(len(input_spectrum) * 0.10))
#     filtered_2 = signal.convolve(input_spectrum, boxcar_2, mode='same')
#     filtered_10 = signal.convolve(input_spectrum, boxcar_10, mode='same')

#     # Step 2: Calculate the difference of gaussians of the two resulting pairs from the filtering step
#     diff_of_gaussians = filters.gaussian_filter1d(filtered_2, sigma=1) - filters.gaussian_filter1d(filtered_10, sigma=1)

#     # Step 3: Take Gaussian difference result with the largest standard deviation and divide the first filtering step result by the second smoothing result to extract the local minima, which are preliminary absorption features
#     if np.std(diff_of_gaussians) > np.std(filtered_2):
#         minima = np.argmin(diff_of_gaussians)
#     else:
#         minima = np.argmin(filtered_2 / filtered_10)

#     # Step 4: Linear interpolation between preliminary features
#     x = np.linspace(0, len(input_spectrum)-1, len(input_spectrum))
#     f = interp1d(x[minima], input_spectrum[minima])
#     interpolated = f(x)

#     # Step 5: Calculate difference between input spectrum and linear interpolation of minimas and find maximum in each of the segments with zero separation
#     diff = input_spectrum - interpolated
#     maxima = np.argmax(diff)

#     # Step 6: Linear interpolation between maxima gives preliminary hull
#     f_max = interp1d(x[maxima], input_spectrum[maxima])
#     preliminary_hull = f_max(x)

#     # Step 7: Iterative check whether the preliminary hull contains zero crossings, if so then add new continuum points
#     zero_crossings = np.where(np.diff(np.sign(diff)))[0]
#     while len(zero_crossings) > 0:
#         maxima = np.append(maxima, zero_crossings)
#         maxima.sort()
#         f_max = interp1d(x[maxima], input_spectrum[maxima])
#         preliminary_hull = f_max(x)
#         diff = input_spectrum - preliminary_hull
#         zero_crossings = np.where(np.diff(np.sign(diff)))[0]

#     # Step 8: Geometric hull
#     geometric_hull = preliminary_hull

#     return geometric_hull


/tmp/ipykernel_35578/1872684754.py:13: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  diff_of_gaussians = filters.gaussian_filter1d(filtered_2,
/tmp/ipykernel_35578/1872684754.py:14: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  sigma=1) - filters.gaussian_filter1d(filtered_10,


201